In [ ]:
import requests #pour faire des requêtes
import lxml.html as lh #pour l'analyse des tags
import pandas as pd #pour structurer les données
import csv 
import numpy as np #pour manipuler les données
import sqlite3 #pour les requetes

In [ ]:
# lien pour recueillir les informations

url = 'https://auvergnerhonealpes.simplon.co/candidatures.html'
uri_pages = '?page=' #pages
nbPages = 1 # page de départ
tags = ['//h2[@class="card-title"]', \
        '//div[@class="card-content"]', \
        '//span[@class="day"]', \
        '//span[@class="month"]', \
        '//div[@class="year"]', \
        '//div[@class="card-session-info calendar"]', \
        '//div[@class="card-session-infos"]', \
        
       ]

cols = ['Nom', 'Lieu', 'Jour', 'Mois', 'Annee', 'Debut', 'Information'] #nom colonnes pour information formation
 
page = requests.get(url) # faire la requete pour recupérer l'information
doc = lh.fromstring(page.content) # lire le contenu de la requête

Intitulé de chaque tag

card-title = intitulé de la formation
card-content= le lieu et comment la formation est pris en charge
day = le jour  ou aura lieu cette formation
month = le mois ou aura lieu cette formation
card-session-infos = le mois ainsi que l'année de début
card-session-info calendar = le mois ainsi que l'année de début, la durée de la formation, le lieu

### Fonction pour recueillir les éléments de chaque page :

In [ ]:
def getPage(url): 
    page = requests.get(url)
    doc = lh.fromstring(page.content)
 
    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))
 
    # Rassembler les données et alimenter un dataframe
    df_liste = []
    for j in range(len(tags)):#  récupère les infos de chaque tag et mets les dans la colonne correspondant
        #tmp strip supprimer les espaces à droite et à gauche, supprimer les espaces à droite 
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index #récupéré les lignes et les mettre dans la colonne 'key'
        df_liste.append(tmp)#liste des df récupérés
 
    # Crée un dataframe avec une colonne par tag (xpath)
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r') 
        # joindre les éléments de chaque colonne pour créer une ligne de chaque colonne
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
    return liste
 
def getPages(_nbPages, _url): #récupérer les éléments de chaque page
    liste_finale = pd.DataFrame()
    for i in range (_nbPages):
        liste = getPage(_url + uri_pages + str(i+1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)# concaténer les résultats
    return liste_finale
 
liste_totale = getPages(nbPages, url)


[[<Element h2 at 0x21ad4ac2958>, <Element h2 at 0x21ad4ac24f8>, <Element h2 at 0x21ad4ac2b88>, <Element h2 at 0x21ad4ac9688>, <Element h2 at 0x21ad4aca7c8>, <Element h2 at 0x21ad4aca458>, <Element h2 at 0x21ad4acac78>, <Element h2 at 0x21ad4acabd8>, <Element h2 at 0x21ad4acac28>, <Element h2 at 0x21ad4acacc8>, <Element h2 at 0x21ad4acad18>, <Element h2 at 0x21ad4acad68>]]
[[<Element h2 at 0x21ad4ac2958>, <Element h2 at 0x21ad4ac24f8>, <Element h2 at 0x21ad4ac2b88>, <Element h2 at 0x21ad4ac9688>, <Element h2 at 0x21ad4aca7c8>, <Element h2 at 0x21ad4aca458>, <Element h2 at 0x21ad4acac78>, <Element h2 at 0x21ad4acabd8>, <Element h2 at 0x21ad4acac28>, <Element h2 at 0x21ad4acacc8>, <Element h2 at 0x21ad4acad18>, <Element h2 at 0x21ad4acad68>], [<Element div at 0x21ad4acae08>, <Element div at 0x21ad4acae58>, <Element div at 0x21ad4acaea8>, <Element div at 0x21ad4acaef8>, <Element div at 0x21ad4acaf48>, <Element div at 0x21ad4acaf98>, <Element div at 0x21ad4ace048>, <Element div at 0x21ad4ac

                                                  Nom  \
0   Apple Foundation Program - Développement mobil...   
1   Apple Foundation Program - Développement mobil...   
2   Ecole IA Microsoft By Simplon - Développeur.eu...   
3   Concepteur développeur d'applications - 100% D...   
4   Administrateur·trice d’infrastructures sécuris...   
5                                           Refugeeks   
6   Gérer un projet en mobilisant les méthodes agi...   
7   CléA Numérique - Compétences Digitales pour l’...   
8   Gérer un projet en mobilisant les méthodes agi...   
9                             Développer avec JAVA EE   
10  CléA Numérique - Compétences Digitales pour l’...   
11  Développeur  web et web mobile - entreprise XE...   

                                                 Lieu Jour Mois Annee  key  
0                                        Simplon Lyon   14   11  2021    0  
1                                        Simplon Lyon   03   10  2021    1  
2                          

In [ ]:
liste_totale

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key
0,Apple Foundation Program - Développement mobil...,Simplon Lyon,14,11,2021,event\n \n\n Déb...,event\n \n\n Déb...,0
1,Apple Foundation Program - Développement mobil...,Simplon Lyon,03,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,1
2,Ecole IA Microsoft By Simplon - Développeur.eu...,Simplon Grenoble,15,11,2021,event\n \n\n Déb...,event\n \n\n Déb...,2
3,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,3
4,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,14,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,4
5,Refugeeks,Simplon Lyon,21,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,5
6,Gérer un projet en mobilisant les méthodes agi...,CPF\n \n ...,05,11,2021,event\n \n\n Déb...,event\n \n\n Déb...,6
7,CléA Numérique - Compétences Digitales pour l’...,CPF\n \n ...,05,11,2021,event\n \n\n Déb...,event\n \n\n Déb...,7
8,Gérer un projet en mobilisant les méthodes agi...,CPF\n \n ...,29,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,8
9,Développer avec JAVA EE,Simplon Lyon,31,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,9


In [ ]:
liste_totale['Debut']

0     event\n                \n\n                Déb...
1     event\n                \n\n                Déb...
2     event\n                \n\n                Déb...
3     event\n                \n\n                Déb...
4     event\n                \n\n                Déb...
5     event\n                \n\n                Déb...
6     event\n                \n\n                Déb...
7     event\n                \n\n                Déb...
8     event\n                \n\n                Déb...
9     event\n                \n\n                Déb...
10    event\n                \n\n                Déb...
11    event\n                \n\n                Déb...
Name: Debut, dtype: object

Nettoyage de la colonne Debut, ,remplacer les éléments inutiles par des espaces

In [ ]:
liste_totale['Debut'] = liste_totale['Debut'].replace({'event':''}, regex= True)

In [ ]:
liste_totale['Debut'] = liste_totale['Debut'].replace({'\n':''}, regex= True)
liste_totale['Debut'] = liste_totale['Debut'].replace({'Début':''}, regex= True)
liste_totale['Debut'] = liste_totale['Debut'].replace({':':''}, regex= True)

In [ ]:
liste_totale

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key
0,Apple Foundation Program - Développement mobil...,Simplon Lyon,14,11,2021,novembre 2021,event\n \n\n Déb...,0
1,Apple Foundation Program - Développement mobil...,Simplon Lyon,03,10,2021,octobre 2021,event\n \n\n Déb...,1
2,Ecole IA Microsoft By Simplon - Développeur.eu...,Simplon Grenoble,15,11,2021,décembre 2021,event\n \n\n Déb...,2
3,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,10,2021,septembre 2021,event\n \n\n Déb...,3
4,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,14,10,2021,septembre 2021,event\n \n\n Déb...,4
5,Refugeeks,Simplon Lyon,21,10,2021,novembre 2021,event\n \n\n Déb...,5
6,Gérer un projet en mobilisant les méthodes agi...,CPF\n \n ...,05,11,2021,novembre 2021,event\n \n\n Déb...,6
7,CléA Numérique - Compétences Digitales pour l’...,CPF\n \n ...,05,11,2021,novembre 2021,event\n \n\n Déb...,7
8,Gérer un projet en mobilisant les méthodes agi...,CPF\n \n ...,29,10,2021,novembre 2021,event\n \n\n Déb...,8
9,Développer avec JAVA EE,Simplon Lyon,31,10,2021,novembre 2021,event\n \n\n Déb...,9


Nettoyage de la colonne information, supprimer les éléments inutiles

In [ ]:
liste_totale['Information'] = liste_totale['Information'].replace({'event':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({'\n':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({'Début':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({':':''}, regex= True)

In [ ]:
liste_totale

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key
0,Apple Foundation Program - Développement mobil...,Simplon Lyon,14,11,2021,novembre 2021,novembre 202...,0
1,Apple Foundation Program - Développement mobil...,Simplon Lyon,03,10,2021,octobre 2021,octobre 2021...,1
2,Ecole IA Microsoft By Simplon - Développeur.eu...,Simplon Grenoble,15,11,2021,décembre 2021,décembre 202...,2
3,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,10,2021,septembre 2021,septembre 20...,3
4,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,14,10,2021,septembre 2021,septembre 20...,4
5,Refugeeks,Simplon Lyon,21,10,2021,novembre 2021,novembre 202...,5
6,Gérer un projet en mobilisant les méthodes agi...,CPF\n \n ...,05,11,2021,novembre 2021,novembre 202...,6
7,CléA Numérique - Compétences Digitales pour l’...,CPF\n \n ...,05,11,2021,novembre 2021,novembre 202...,7
8,Gérer un projet en mobilisant les méthodes agi...,CPF\n \n ...,29,10,2021,novembre 2021,novembre 202...,8
9,Développer avec JAVA EE,Simplon Lyon,31,10,2021,novembre 2021,novembre 202...,9


Découpage des données pour récupérer les informations liés à la date

In [ ]:
df = liste_totale.copy()
df["duree1"] = df["Information"].str.split().str[3]
df["duree2"] = df["Information"].str.split().str[4]
df["duree3"] = df["Information"].str.split().str[5]
df["duree4"] = df["Information"].str.split().str[6]

In [ ]:
df["Localisation"] = df["Information"].str.split().str[8]

In [ ]:
df = df.drop('Information', axis=1)

In [ ]:
df['Duree'] = df['duree1'] + ' ' + df['duree2'] + ' ' + df['duree3'] + ' ' + df['duree4']

In [ ]:
df = df.drop(['duree1', 'duree2', 'duree3', 'duree4',], axis=1)

In [ ]:
df['datelimite'] = df['Jour'] + '-' + df['Mois'] + '-' + df['Annee']

In [ ]:
df = df.drop(['Jour', 'Mois', 'Annee'], axis=1)

Savoir si oui ou non distanciel, création de la colonne pour savoir

In [ ]:
# si DISTANCIEL == True, alors écrit 'Distanciel'
df['methode'] = np.where(df['Nom'].str.contains("DISTANCIEL")== True, 'Distanciel', 'Présentiel')

Création de condition pour savoir le financement de la formation

In [ ]:
# si l'un de ces mots est contenu dans la colonne lieu, inscris là dans la colonne paiement

conditionlist = [
    (df['Lieu'].str.contains("Alternance")== True) ,
    (df['Lieu'].str.contains("CPF")== True),
    (df['Lieu'].str.match("^[Simplon Lyon].*")== True)]
choicelist = ['Alternance', 'CPF', 'Gratuit']
df['Paiement'] = np.select(conditionlist, choicelist, default='Not Specified')

In [ ]:
df['Lieu'] = df['Lieu'].replace({'Alternance':''}, regex= True)
df['Lieu'] = df['Lieu'].replace({'\n':''}, regex= True)
df['Lieu'] = df['Lieu'].replace({'CPF':''}, regex= True)

In [ ]:
df

,Nom,Lieu,Debut,key,Localisation,Duree,datelimite,methode,Paiement
0,Apple Foundation Program - Développement mobil...,Simplon Lyon,novembre 2021,0,Auvergne-Rhône-Alpes,inférieur à 3 mois,14-11-2021,Présentiel,Gratuit
1,Apple Foundation Program - Développement mobil...,Simplon Lyon,octobre 2021,1,Auvergne-Rhône-Alpes,inférieur à 3 mois,03-10-2021,Présentiel,Gratuit
2,Ecole IA Microsoft By Simplon - Développeur.eu...,Simplon Grenoble,décembre 2021,2,Auvergne-Rhône-Alpes,au-delà de 7 mois,15-11-2021,Présentiel,Gratuit
3,Concepteur développeur d'applications - 100% D...,AUVERG...,septembre 2021,3,Auvergne-Rhône-Alpes,au-delà de 7 mois,21-10-2021,Distanciel,Alternance
4,Administrateur·trice d’infrastructures sécuris...,AUVERG...,septembre 2021,4,Auvergne-Rhône-Alpes,au-delà de 7 mois,14-10-2021,Distanciel,Alternance
5,Refugeeks,Simplon Lyon,novembre 2021,5,Auvergne-Rhône-Alpes,inférieur à 3 mois,21-10-2021,Présentiel,Gratuit
6,Gérer un projet en mobilisant les méthodes agi...,Simplo...,novembre 2021,6,Auvergne-Rhône-Alpes,inférieur à 10 jours,05-11-2021,Présentiel,CPF
7,CléA Numérique - Compétences Digitales pour l’...,Simplo...,novembre 2021,7,Auvergne-Rhône-Alpes,inférieur à 10 jours,05-11-2021,Présentiel,CPF
8,Gérer un projet en mobilisant les méthodes agi...,Simplo...,novembre 2021,8,Auvergne-Rhône-Alpes,inférieur à 10 jours,29-10-2021,Présentiel,CPF
9,Développer avec JAVA EE,Simplon Lyon,novembre 2021,9,Auvergne-Rhône-Alpes,inférieur à 7 mois,31-10-2021,Présentiel,Gratuit


Création du csv

In [ ]:
df.to_csv('formation_simplon.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

Exportation des données dans la base de données

In [ ]:
#On créer la base de donnée
con = sqlite3.connect('simplon_bd.db')
#On fixe le curseur 
c = con.cursor()

In [ ]:
#On ajoute les tables et les données dans la base de donnée
df.to_sql('simplon', con, if_exists='append', index= False)